In [26]:
from pygalfitm import PyGalfitm
from pygalfitm.VOs import splus
from pygalfitm.auxiliars import string_times_x, get_dims, get_exptime, unpack_file
from pygalfitm.psf import make_psf

import splusdata

import os

In [29]:
name = "ari_test"

ra = 52.430082897775755	
dec = -33.557141932864596
cut_size = 200
bands = ["I", "R", "G"]

DATA_FOLDER = "data/"
OUTPUT_FOLDER = "outputs/"

In [31]:
conn = splusdata.connect("gustavo", "asdflkjh")

You have access to internal data


In [32]:
pyg = PyGalfitm() 

In [33]:
input_images = ""
psf_images = ""
filters = ""

for band in bands:
    try:
        conn.get_cut(ra, dec, 200, band, filepath=os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits'))
    except Exception as e:
        print(e)
    
    try:
        unpack_file(os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits.fz'))
    except Exception as e:
        print(e)
        print("Make sure you have fpack (cfitsio) in your system alias.")
    
    make_psf(os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits'), outfile=os.path.join(DATA_FOLDER, f'psf_{name}_{band.lower()}.fits'))

    input_images += "," + os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits')
    psf_images += "," + os.path.join(DATA_FOLDER, f'psf_{name}_{band.lower()}.fits')

    filters += "," + str(band).lower()

input_images = input_images[1:]
psf_images = psf_images[1:]
filters = filters[1:]

File data/ari_test_i.fits.fz already exists. If you mean to replace it then use the argument "overwrite=True".
File data/ari_test_r.fits.fz already exists. If you mean to replace it then use the argument "overwrite=True".
File data/ari_test_g.fits.fz already exists. If you mean to replace it then use the argument "overwrite=True".


In [34]:
import pandas as pd

df = pd.read_csv("pygalfitm/VOs/splus_idr4_zps.csv")

In [35]:
from astropy.io.fits import getheader

header = getheader(os.path.join(DATA_FOLDER, name + "_r.fits"))
field = header['OBJECT']

In [36]:
zps = ""

for band in bands:
    zps += "," + str(df[df['Field'] == field][f'ZP_{band.lower().replace("f", "J0")}'].values[0])

zps = zps[1:]

In [37]:
zps

'23.344,23.535,23.49'

In [38]:
pyg.set_base({
    "A": input_images,
    "A1": filters,
    "B": os.path.join(OUTPUT_FOLDER, name + "ss.fits"),
    "C": "none",
    "D": psf_images,
    "A2": "7625,6231,4770",
    "H": f"1   {cut_size}  1   {cut_size}",
    "I": f"{cut_size} {cut_size}",
    "J": zps,
    "K": "0.55 0.55"
})

In [39]:
pyg.print_base()

A) data/ari_test_i.fits,data/ari_test_r.fits,data/ari_test_g.fits # Input data image (FITS file)
A1) i,r,g                            # Nick names (band labels) 
A2) 7625,6231,4770                   # Effective wavelenghts
B) outputs/ari_testss.fits          # Output data image block
C) none                             # Sigma image name (made from data if blank or 'none')
D) data/psf_ari_test_i.fits,data/psf_ari_test_r.fits,data/psf_ari_test_g.fits # Input PSF image and (optional) diffusion kernel
E) 1                                # PSF fine sampling factor relative to data 
F) none                             # Bad pixel mask (FITS image or ASCII coord list)
G) none                             # File with parameter constraints (ASCII file) 
H) 1   200  1   200                 # Image region to fit (xmin xmax ymin ymax)
I) 200 200                          # Size of the convolution box (x y)
J) 23.344,23.535,23.49              # Magnitude photometric zeropoint
K) 0.55 0.55           

In [40]:
axis_ratios, effective_rs, position_angles, mags = splus.get_sersic_splus(conn, ra, dec, bands)

finished
finished
finished


In [13]:
pyg.set_component("sersic", "1", string_times_x(cut_size / 2, 3))
pyg.set_component("sersic", "2", string_times_x(cut_size / 2, 3))
pyg.set_component("sersic", "3", mags)
pyg.set_component("sersic", "4", effective_rs)
pyg.set_component("sersic", "5", string_times_x("4", 3))
pyg.set_component("sersic", "9", axis_ratios)
pyg.set_component("sersic", "10", position_angles)

pyg.print_component("sersic")

1) 100.0,100.0,100.0                   1    band       # Position x [pixel]
2) 100.0,100.0,100.0                   1    band       # Position y [pixel]
3) 14.022123,14.107952,14.922325       3    band       # Integrated magnitude
4) 25.440218,30.087149,24.733025       2    band       # R_e (effective radius) [pix]
5) 4,4,4                               2    band       # Sersic index n (de Vaucouleurs n=4)
9) 0.6433851,0.7046503,0.7636552       1    band       # Axis ratio (b/a)
10) 15.269562,23.058279,32.80171        1    band       # Position angle (PA) [deg: Up=0, Left=90]
Z) 0                                                   # Skip this model in output image? (yes=1, no=0)


In [70]:
pyg = PyGalfitm()

In [75]:
pyg.set_component("sersic", {
    "1": (string_times_x(cut_size / 2, 3), 1, "band" ),
    "2": string_times_x(cut_size / 2, 3), 
    "3": mags, 
    "4": effective_rs,
    "5": string_times_x("4", 3),
    "9": axis_ratios,
    "10": position_angles
})

In [76]:
pyg.set_component("sersic", {
    "1": string_times_x(cut_size / 2, 3),
    "2": string_times_x(cut_size / 2, 3), 
    "3": mags, 
    "4": effective_rs,
    "5": string_times_x("4", 3),
    "9": axis_ratios,
    "10": position_angles
})

pyg.set_component("sersic", {
    "1": 2,
    "2": 3, 
    "3": 2, 
    "4": 3,
    "5": 1,
    "9": 4,
    "10": 5
}, column=2)

pyg.set_component("sersic", {
    "1": "cheb",
    "2": "cheb", 
    "3": "cheb", 
    "4": "cheb",
    "5": "cheb",
    "9": "cheb",
    "10": "cheb"
}, column=3)

pyg.print_component("sersic")

1) 100.0,100.0,100.0                   2    cheb       # Position x [pixel]
2) 100.0,100.0,100.0                   3    cheb       # Position y [pixel]
3) 14.022123,14.107952,14.922325       2    cheb       # Integrated magnitude
4) 25.440218,30.087149,24.733025       3    cheb       # R_e (effective radius) [pix]
5) 4,4,4                               1    cheb       # Sersic index n (de Vaucouleurs n=4)
9) 0.6433851,0.7046503,0.7636552       4    cheb       # Axis ratio (b/a)
10) 15.269562,23.058279,32.80171        5    cheb       # Position angle (PA) [deg: Up=0, Left=90]
Z) 0                                                   # Skip this model in output image? (yes=1, no=0)


In [77]:
pyg.print_component("sersic")

1) 100.0,100.0,100.0                   2    cheb       # Position x [pixel]
2) 100.0,100.0,100.0                   3    cheb       # Position y [pixel]
3) 14.022123,14.107952,14.922325       2    cheb       # Integrated magnitude
4) 25.440218,30.087149,24.733025       3    cheb       # R_e (effective radius) [pix]
5) 4,4,4                               1    cheb       # Sersic index n (de Vaucouleurs n=4)
9) 0.6433851,0.7046503,0.7636552       4    cheb       # Axis ratio (b/a)
10) 15.269562,23.058279,32.80171        5    cheb       # Position angle (PA) [deg: Up=0, Left=90]
Z) 0                                                   # Skip this model in output image? (yes=1, no=0)


In [20]:
pyg.active_components = ["sersic"]

In [21]:
pyg.write_feedme()

In [22]:
_ = pyg.run()

b'\nGALFITM Version galfitm-1.4.4\n(based on GALFIT Version 3.0.5)\n\n\n#  Input menu file: galfit.feedme\n\n================================================================================\n# IMAGE and GALFIT CONTROL PARAMETERS\nA) data/ari_test_i.fits,data/ari_test_r.fits,data/ari_test_g.fits      # Input data image (FITS file)\nA1) i,r,g              # Band labels\nA2) 7625.00,6231.00,4770.00     # Band wavelengths\nB) outputs/ari_testss.fits      # Output data image block\nC) none,none,none 0.000   # Sigma image and min. sigma factor (made from data if blank or "none") \nD) data/psf_ari_test_i.fits,data/psf_ari_test_r.fits,data/psf_ari_test_g.fits #        # Input PSF image and (optional) diffusion kernel\nE) 1                   # PSF fine sampling factor relative to data \nF) none,none,none      # Bad pixel mask (FITS image or ASCII coord list)\nG) none                # File with parameter constraints (ASCII file) \nH) 1    200  1    200  # Image region to fit (xmin xmax ymin ymax

In [53]:
type((1, 2, 3))

tuple

In [69]:
import os

class PyGalfitm:
    def __init__(self):
        self.feedme_path = "galfit.feedme"
        self.executable = "./new_tests/galfitm-1.4.4-osx"

        self.base = {
            "A": {"value": "", "comment": "Input data image (FITS file)"},
            "A1": {"value": "g, r, i", "comment": "Nick names (band labels) "},
            "A2": {"value": "4770, 6231, 7625", "comment": "Effective wavelenghts"}, 
            "B": {"value": "4770, 6231, 7625", "comment": "Output data image block"}, 
            "C": {"value": "", "comment": "Sigma image name (made from data if blank or 'none')"},
            "D": {"value": "", "comment": "Input PSF image and (optional) diffusion kernel"}, 
            "E": {"value": "1", "comment": "PSF fine sampling factor relative to data "}, 
            "F": {"value": "none", "comment": "Bad pixel mask (FITS image or ASCII coord list)"}, 
            "G": {"value": "none", "comment": "File with parameter constraints (ASCII file) "},
            "H": {"value": "1    200  1  200", "comment": "Image region to fit (xmin xmax ymin ymax)"},
            "I": {"value": "200  200", "comment": "Size of the convolution box (x y)"},
            "J": {"value": "0,0,0", "comment": "Magnitude photometric zeropoint"},
            "K": {"value": "0.55  0.55", "comment": "Plate scale (dx dy)   [arcsec per pixel]"},
            "O": {"value": "regular", "comment": "Display type (regular, curses, both)"}, 
            "P": {"value": "0", "comment": "Choose: 0=optimize, 1=model, 2=imgblock, 3=subcomps"}, 
            "U": {"value": "0", "comment": ""}
        }

        self.components_config = {
            "sky" : {
                "1": {"col1": "BKGG,BKGR,BKGI", "col2": "0", "col3": "band", "comment": "Sky background at center of fitting region [ADUs]"},
                "2": {"col1": "0,0,0", "col2": "0", "col3": "band", "comment": "dsky/dx (sky gradient in x) [ADUs/pix]"},
                "3": {"col1": "0,0,0", "col2": "0", "col3": "band", "comment": "dsky/dy (sky gradient in y) [ADUs/pix]"},
                "Z": {"col1": "0", "col2": "", "col3": "", "comment": "Skip this model in output image? (yes=1, no=0)"}
            },
            "sersic" : {
                "1":  {"col1": "200.0,200.0,200.0", "col2": "1", "col3": "band", "comment": "Position x [pixel]"},
                "2":  {"col1": "200.0,200.0,200.0", "col2": "1", "col3": "band", "comment": "Position y [pixel]"},
                "3":  {"col1": "0,0,0", "col2": "3", "col3": "band", "comment": "Integrated magnitude"},
                "4":  {"col1": "0,0,0", "col2": "2", "col3": "band", "comment": "R_e (effective radius) [pix]"},
                "5":  {"col1": "4", "col2": "2", "col3": "band", "comment": "Sersic index n (de Vaucouleurs n=4)"},
                "9":  {"col1": "0,0,0", "col2": "1", "col3": "band", "comment": "Axis ratio (b/a)"},
                "10": {"col1": "0,0,0", "col2": "1", "col3": "band", "comment": "Position angle (PA) [deg: Up=0, Left=90]"},
                "Z":  {"col1": "0", "col2": "", "col3": "", "comment": "Skip this model in output image? (yes=1, no=0)"}
            },
            "expdisk" : {
                "1":  {"col1": "300", "col2": "1", "col3": "band", "comment": "Position x [pixel]"},
                "2":  {"col1": "357.4", "col2": "1", "col3": "band", "comment": "Position y [pixel]"},
                "3":  {"col1": "0,0,0", "col2": "3", "col3": "band", "comment": "Integrated magnitude"},
                "4":  {"col1": "0,0,0", "col2": "2", "col3": "band", "comment": "R_s (disk scale lengths) [pix]"},
                "9":  {"col1": "0,0,0", "col2": "1", "col3": "band", "comment": "Axis ratio (b/a)"},
                "10": {"col1": "0,0,0", "col2": "1", "col3": "band", "comment": "Position angle (PA) [deg: Up=0, Left=90]"},
                "Z":  {"col1": "0", "col2": "", "col3": "", "comment": "Skip this model in output image? (yes=1, no=0)"}
            },
            "moffat" : {
                "1":  {"col1": "300", "col2": "1", "col3": "band", "comment": "Position x [pixel]"},
                "2":  {"col1": "357.4", "col2": "1", "col3": "band", "comment": "Position y [pixel]"},
                "3":  {"col1": "0,0,0", "col2": "3", "col3": "band", "comment": "Total magnitude"},
                "4":  {"col1": "0,0,0", "col2": "2", "col3": "band", "comment": "FWHM"},
                "5":  {"col1": "0,0,0", "col2": "2", "col3": "band", "comment": "powerlaw"},
                "9":  {"col1": "0,0,0", "col2": "1", "col3": "band", "comment": "Axis ratio (b/a)"},
                "10": {"col1": "0,0,0", "col2": "1", "col3": "band", "comment": "Position angle (PA) [deg: Up=0, Left=90]"},
                "Z":  {"col1": "0", "col2": "", "col3": "", "comment": "Skip this model in output image? (yes=1, no=0)"}
            },
            "ferrer" : {
                "1":  {"col1": "300", "col2": "1", "col3": "band", "comment": "Position x [pixel]"},
                "2":  {"col1": "357.4", "col2": "1", "col3": "band", "comment": "Position y [pixel]"},
                "3":  {"col1": "0,0,0", "col2": "3", "col3": "band", "comment": "Central surface brghtness [mag/arcsec^2]"},
                "4":  {"col1": "0,0,0", "col2": "2", "col3": "band", "comment": "Outer truncation radius  [pix]"},
                "5":  {"col1": "0,0,0", "col2": "2", "col3": "band", "comment": "Alpha (outer truncation sharpness) "},
                "6":  {"col1": "0,0,0", "col2": "2", "col3": "band", "comment": "Beta (central slope)"},
                "9":  {"col1": "0,0,0", "col2": "1", "col3": "band", "comment": "Axis ratio (b/a)"},
                "10": {"col1": "0,0,0", "col2": "1", "col3": "band", "comment": "Position angle (PA) [deg: Up=0, Left=90]"},
                "Z":  {"col1": "0", "col2": "", "col3": "", "comment": "Skip this model in output image? (yes=1, no=0)"}
            },
            "psf" : {
                "1": {"col1": "0,0,0", "col2": "0", "col3": "band", "comment": "position x [pixel]"},
                "2": {"col1": "0,0,0", "col2": "0", "col3": "band", "comment": "position y [pixel]"},
                "3": {"col1": "0,0,0", "col2": "0", "col3": "band", "comment": "total magnitude "},
                "Z": {"col1": "0", "col2": "", "col3": "", "comment": "Skip this model in output image? (yes=1, no=0)"}
            }
        }

        self.components = [
            "sersic",
            "expdisk", 
            "moffat",
            "ferrer",
            "psf",
            "sky"
        ]

        self.active_components = []


    def activate_components(self, component_s = None):
        if component_s is None:
            self.activate_components = []
        if isinstance(component_s, list):
            for comp in component_s:
                if comp in self.components:
                    if comp not in self.active_components:
                        self.active_components.append(comp)
                else:
                    raise Exception(f"Not valid component - {comp}")
        else:
            if component_s in self.components:
                if component_s not in self.active_components:
                    self.active_components.append(component_s)
            else:
                raise Exception(f"Not valid component - {component_s}")


    def set_base(self, item, value=""):
        if isinstance(item, dict):
            for i in item:
                self.base[i]["value"] = str(item[i])
        else:
            if item in self.base:
                self.base[item]["value"] = str(value)
            else:
                raise KeyError("Parameter not found in galfitm feedme base config.")
        
    def set_component(self, component, item = None, value = None, column = 1):
        if column in [1, 2, 3]:
            column = 'col' + str(column)
        else:
            raise Exception("Column not valid.")
        if component in self.components_config:
            
            if isinstance(item, dict):
                for i in item:
                    if isinstance(item[i], tuple):
                        if len(item[i]) > 3:
                            raise Exception("Tuple not valid " + str(item[i]))
                        for key, val in enumerate(item[i]):
                            self.components_config[component][i]["col" + str(key + 1)] = str(val)
                    else:
                        self.components_config[component][i][column] = str(item[i])
            else:
                if item in self.components_config[component]:
                    self.components_config[component][item][column] = str(value)
        else:
            raise KeyError("Component not found.")


    def write_feedme(self, feedme_path = None):
        if feedme_path is None:
            feedme_path = self.feedme_path
        self.write_base(feedme_path)

        for component in self.active_components:
            self.write_component(component, feedme_path)
    
    def print_component(self, component):
        config = self.components_config[component]
        if component in self.components_config:
            for i in self.components_config[component]:
                final = i + ") " + config[i]['col1'].ljust(35) + " " + config[i]['col2'].ljust(5) + config[i]['col3'].ljust(10) + " # " + config[i]['comment']
                print(final)
        else:
            raise KeyError("Component not found.")
    
    def print_base(self):
        for param in self.base:
            final = str(param) + ") " + str(self.base[param]["value"]).ljust(32) + " # " + str(self.base[param]["comment"])
            print(final)


    def write_base(self, feedme_path = None):
        if feedme_path is None:
            feedme_path = self.feedme_path
        file = open(feedme_path, "w")
        for param in self.base:
            final = str(param) + ") " + str(self.base[param]["value"]).ljust(32) + " # " + str(self.base[param]["comment"]) + "\n"
            file.write(final)
        file.close()
    
    def write_component(self, component_name, feedme_path = None):
        if feedme_path is None:
            feedme_path = self.feedme_path

        if component_name in self.active_components:
            config = self.components_config[component_name]
            f = open(feedme_path, "a")
            f.write("\n\n\n")

            f.write("0) " + component_name + "\n")
            for i in self.components_config[component_name]:
                final = i + ") " + config[i]['col1'].ljust(35) + " " + config[i]['col2'].ljust(5) + config[i]['col3'].ljust(10) + " # " + config[i]['comment'] + "\n"
                f.write(final)

            f.close()
    
    def run(self):
        import subprocess
        output = subprocess.check_output(f'{self.executable} {self.feedme_path}', shell=True).decode("UTF-8")
        
        return output